Import libraries

In [117]:
import sqlite3
import pandas as pd

Connecting database

In [118]:
conn = sqlite3.connect('/home/nsm-07/Downloads/S30 ETL Assignment v22-01 (1) (1)/S30 ETL Assignment.db')
print("Opened database successfully")

Opened database successfully


In [119]:
# create cursor connections for required tabels by fetching data
customers_tabel = conn.execute("SELECT * from customers")
items_tabel = conn.execute("SELECT * from items")
sales_tabel = conn.execute("SELECT * from sales")
orders_tabel = conn.execute("SELECT * from orders")

1. SQL solution

In [120]:
# sql commands to get required report
# 1. connect with database trough command line
# 2. declare the headers for csv file
# 3. declare the file format mode.
# 4. declare the separator
# 5. declare the output file name
# 6. execute the query to get the final report and create csv file

In [121]:
# required commands for above procedure
# 1. sqlite3> sqlite3 "database full path"
# 2. sqlite3> .headers on
# 3. sqlite3> .mode csv
# 4. sqlite3> .separator ;
# 5. sqlite3> .output final_report.csv
# 6. sqlite3> "select c.customer_id as Customer, c.age as Age, i.item_name as Item, sum(o.quantity) as Quantity from customers as c inner join sales as s on c.customer_id=s.customer_id inner join orders as o on s.sales_id=o.sales_id inner join items as i on o.item_id==i.item_id where Age BETWEEN 18 and 35 and Quantity is not null and Quantity!=0 group by c.customer_id,i.item_name order by Customer"

Sample output using python and sqlite3

In [122]:
sql_report = conn.execute("select c.customer_id as Customer, c.age as Age, i.item_name as Item, sum(o.quantity) as Quantity from customers as c inner join sales as s on c.customer_id=s.customer_id inner join orders as o on s.sales_id=o.sales_id inner join items as i on o.item_id==i.item_id where Age BETWEEN 18 and 35 and Quantity is not null and Quantity!=0 group by c.customer_id,i.item_name order by Customer")
final_report = []
for records in sql_report:
    d = {}
    d["Customer"] = records[0]
    d["Age"] = records[1]
    d["Item"] = records[2]
    d["Quantity"] = records[3]
    final_report.append(d)
final_report_df = pd.DataFrame(final_report)
final_report_df

,Customer,Age,Item,Quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
65,97,23,y,2
66,97,23,z,3
67,100,30,x,1
68,100,30,y,1


2. Python pandas solution

In [123]:
# Create customers DataFrame by getting records from customers_tabel
customers = []
for i in customers_tabel:
    c = {}
    c["customer_id"] = i[0]
    c["age"] = i[1]
    customers.append(c)
customers_df = pd.DataFrame(customers)
customers_df.head()

,customer_id,age
0,1,38
1,2,43
2,3,13
3,4,39
4,5,18


In [124]:
# Create items DataFrame by getting records from items
items = []
for i in items_tabel:
    c = {}
    c["item_id"] = i[0]
    c["item_name"] = i[1]
    items.append(c)
items_df = pd.DataFrame(items)
items_df.head()

,item_id,item_name
0,1,x
1,2,y
2,3,z


In [125]:
# Create sales DataFrame by getting records from sales
sales = []
for i in sales_tabel:
    c = {}
    c["sales_id"] = i[0]
    c["customer_id"] = i[1]
    sales.append(c)
sales_df = pd.DataFrame(sales)
sales_df.head()

,sales_id,customer_id
0,1,10
1,2,85
2,3,42
3,4,93
4,5,75


In [126]:
# Create orders DataFrame by getting records from orders
orders = []
for i in orders_tabel:
    c = {}
    c["order_id"] = i[0]
    c["sales_id"] = i[1]
    c["item_id"] = i[2]
    c["quantity"] = i[3]
    orders.append(c)
orders_df = pd.DataFrame(orders)
orders_df.head()

,order_id,sales_id,item_id,quantity
0,1,1,1,2.0
1,2,1,2,NaN
2,3,1,3,1.0
3,4,2,1,1.0
4,5,2,2,1.0


In [127]:
# joining customers and sales dataframes to combine the columns
customer_sales = pd.merge(sales_df,customers_df, how="inner")
customer_sales.head()

,sales_id,customer_id,age
0,1,10,76
1,57,10,76
2,111,10,76
3,124,10,76
4,210,10,76


In [128]:
# get customer id based on their age filter from 18 to 35 from customers_df
cust_id = customers_df[(customers_df["age"] >= 18) & (customers_df["age"] <= 35)]["customer_id"].tolist()

In [129]:
# get sales id based on their age filter from 18 to 35 from sales
sales_ids = sales_df[(sales_df["customer_id"].isin(cust_id))]["sales_id"].tolist()
print(sales_ids)

[8, 14, 17, 18, 28, 29, 36, 38, 39, 42, 43, 44, 48, 51, 53, 54, 59, 60, 61, 63, 64, 66, 79, 87, 95, 96, 98, 100, 101, 107, 108, 114, 116, 121, 123, 136, 139, 141, 142, 144, 145, 147, 153, 155, 159, 164, 171, 173, 174, 177, 178, 183, 185, 201, 217, 218, 231, 238, 239, 241, 250, 253, 254, 257, 258, 259, 264, 271, 273, 278, 279, 287, 292, 294, 296, 298, 302, 305, 309, 312, 314, 315, 318, 321, 322, 326, 327, 329, 337, 345, 348, 349, 356, 357, 360, 363, 364, 366, 372, 380, 386, 389, 403, 405, 409, 413, 423, 425, 429, 434, 437, 439, 443, 445, 451, 453, 456, 457, 458, 461, 463, 469, 474, 482, 492, 496, 497, 500]


In [130]:
# get orders details based on customer age filter and sales ids
orders_details = orders_df[(orders_df["sales_id"].isin(sales_ids))]
orders_details.head()

,order_id,sales_id,item_id,quantity
21,22,8,1,NaN
22,23,8,2,NaN
23,24,8,3,NaN
39,40,14,1,1.0
40,41,14,2,1.0


In [131]:
# joininb orders_details, customer_sales dataframes to get 
order_details_with_custifo = pd.merge(orders_details, customer_sales, how='inner')
order_details_with_custifo.head()

,order_id,sales_id,item_id,quantity,customer_id,age
0,22,8,1,NaN,28,21
1,23,8,2,NaN,28,21
2,24,8,3,NaN,28,21
3,40,14,1,1.0,16,19
4,41,14,2,1.0,16,19


In [132]:
# join items dataframe and order_details_with_custifo to combine required output resulted columns
order_details_with_custifo = pd.merge(order_details_with_custifo, items_df, how='inner')

In [133]:
# remove null value records and quantity with "0" to avoid inconsistant analysis
final_report = order_details_with_custifo[~(order_details_with_custifo["quantity"].isnull())&~(order_details_with_custifo["quantity"]==0)].groupby(["customer_id","age","item_name"])["quantity"].sum().reset_index()
final_report

,customer_id,age,item_name,quantity
0,5,18,x,3.0
1,5,18,y,3.0
2,5,18,z,4.0
3,7,30,x,14.0
4,7,30,y,19.0
...,...,...,...,...
65,97,23,y,2.0
66,97,23,z,3.0
67,100,30,x,1.0
68,100,30,y,1.0


In [135]:
# conver dataframe to csv
final_report_df.to_csv("final_report.csv",sep=';')